# PydanticOuputParser

In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response
from pydantic import Field

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [6]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

prompt = PromptTemplate.from_template("다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer)

중요한 내용 요약:

- 발신자: 김철수 (바이크코퍼레이션 상무)
- 수신자: 이은채 (Teddy International)
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 요청 사항: ZENESIS 모델에 대한 상세 브로슈어 (기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서 만남 제안
- 목적: 협력 가능성 논의 및 유통 전략, 마케팅 계획 구체화

In [8]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [9]:
# instruction을 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [10]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [11]:
# chain을 생성
chain = prompt | llm

In [12]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해주세요."
    }
)

# 결과는 JSON형태로 출력
output = stream_response(response, return_output=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무가 이은채 대리에게 'ZENESIS' 자전거에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 함.",
  "date": "1월 15일 오전 10시"
}
```

In [13]:
# PydanticOutputParser를 사용하여 결과를 파싱
structed_output = parser.parse(output)
print(structed_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="김철수 상무가 이은채 대리에게 'ZENESIS' 자전거에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 함." date='1월 15일 오전 10시'


# parser가 추가된 체인 생성

In [14]:
# 출력 파서를 추가하여 전체 체인을 구성
chain = prompt | llm | parser

In [16]:
# chain을 실행하고 결과를 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요."
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 'ZENESIS'에 대한 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.", date='1월 15일 오전 10시')

# with_structured_output()

In [17]:
llm_with_structered = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
).with_structured_output(EmailSummary)

In [19]:
# invoke() 함수를 호출하여 결과를 출력
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='이은채', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리에게 바이크코퍼레이션과 'ZENESIS' 자전거의 유통 협력에 대해 논의하고자 미팅을 제안하며, 제품에 대한 상세한 브로슈어 요청.", date='2024-01-08')